In [5]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [7]:
train_data = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_load= DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_load = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [8]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
net = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optim = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
epochs = 20

In [12]:
for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{epochs}', end=' | ')

    running_loss = 0.0

    for i, data in enumerate(train_load):
        inputs, labels = data

        optim.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optim.step()

        running_loss += loss.item()
    print(f'Batch: {i+1}, Loss: {running_loss / (i+1):.3f}')
print('Training complete')

correct = 0
total = 0

with torch.no_grad():
    for data in test_load:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Epoch: 1/20 | Batch: 1563, Loss: 2.151
Epoch: 2/20 | Batch: 1563, Loss: 1.699
Epoch: 3/20 | Batch: 1563, Loss: 1.525
Epoch: 4/20 | Batch: 1563, Loss: 1.418
Epoch: 5/20 | Batch: 1563, Loss: 1.328
Epoch: 6/20 | Batch: 1563, Loss: 1.253
Epoch: 7/20 | Batch: 1563, Loss: 1.184
Epoch: 8/20 | Batch: 1563, Loss: 1.130
Epoch: 9/20 | Batch: 1563, Loss: 1.079
Epoch: 10/20 | Batch: 1563, Loss: 1.033
Epoch: 11/20 | Batch: 1563, Loss: 0.990
Epoch: 12/20 | Batch: 1563, Loss: 0.950
Epoch: 13/20 | Batch: 1563, Loss: 0.912
Epoch: 14/20 | Batch: 1563, Loss: 0.873
Epoch: 15/20 | Batch: 1563, Loss: 0.840
Epoch: 16/20 | Batch: 1563, Loss: 0.808
Epoch: 17/20 | Batch: 1563, Loss: 0.778
Epoch: 18/20 | Batch: 1563, Loss: 0.747
Epoch: 19/20 | Batch: 1563, Loss: 0.722
Epoch: 20/20 | Batch: 1563, Loss: 0.695
Training complete
Accuracy on test set: 67.80%


In [16]:
test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

def predict_image(image_path):
    image = Image.open(image_path)
    image = test_transform(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        output = net(image)
        _, predicted = torch.max(output, 1)
    
    return class_names[predicted.item()]

image_list = ['image.png']
for img_path in image_list:
    prediction = predict_image(img_path)
    print(f'Predicted class for {img_path}: {prediction}')

Predicted class for image.png: plane
